In [1]:
#Code X: Run a baseline model (Logistic Regression) to establish a performance benchmark using linear relationships and features from the CMS OEP PUF dataset.
#This provides a comparative reference point for future models (K-Means Clustering, XGBoost), which will incorporate non-linear clustering interactions and
#external SVI data to improve predictive accuracy.

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, roc_auc_score
import pandas as pd
import numpy as np

In [ ]:
#Read output from [2] "_featureengineering.ipynb"
master_df = pd.read_excel('output/master_df_with_features.xlsx')

In [4]:
#Define the Target (y), using the Top 25% cutoff in Access Disruptuion Risk Index as threshold for "high risk"
cutoff_baseline = master_df['Access_Disruption_Risk_Index'].quantile(0.75)
master_df['Target_Baseline'] = (master_df['Access_Disruption_Risk_Index'] > cutoff_baseline).astype(int)

In [5]:
feature_cols = [                      # Note to self: reflects final_features_to_cluster
    # --- PLAN MIX ---
    'feat_pct_bronze',                # % bronze enrollees
    'feat_pct_gold',                  # % gold enrollees
    #'plan_value_density',            # %gold enrollees divided by (%gold + %bronze enrollees)

    # --- INCOME LEVEL ---
    'feat_pct_FPL_100_150',         # % enrollees 100-150% FPL
    'feat_pct_FPL_150_250',         # % enrollees 150-250% FPL
    'feat_pct_FPL_250_400',         # % enrollees 250-400% FPL
    'feat_pct_FPL_400_plus',       # % enrollees 400%+ FPL

    # --- COST & VALUE ---
    'avg_premium_before_APTC',          # avg premium before APTC in dollars
    'feat_subsidy_coverage_ratio',      # (avg premium (before) - avg premium (after))/avg premium (before)
    
    # --- MARKET BEHAVIOR ---
    'feat_net_attrition',               # net attrition
    'feat_market_newness',              # % of new enrollees
    'feat_passive_renewal',             # % of enrollees with automatic/passive renewal
    
    # --- DEMOGRAPHICS ---
    #'feat_pct_age_18_34',              # % of enrollees 18-34
    'feat_pct_age_55_over',             # % of enrollees 55+yo
    'feat_pct_black',                   # % of enrollees who are African American
    'feat_pct_hispanic',                # % of enrollees who are Hispanic
    'feat_pct_asian',                   # % of enrollees who are Asian
    'feat_pct_aian',                    # % of enrollees who are American Indian/Alaska Native
    #'feat_pct_white'                   # % of enrollees who are White; exclude from clustering but then include when evaluating clusters
]

In [6]:
# DATA VALIDATION, assign X and y
# Verify columns exist
missing_cols = [c for c in feature_cols if c not in master_df.columns]
if missing_cols:
    print(f"⚠️ Missing columns in dataframe: {missing_cols}")
    # Remove missing to allow script to run partially
    feature_cols = [c for c in feature_cols if c in master_df.columns]

X = master_df[feature_cols]

# Data Validation (ensure no NaNs)
if X.isnull().values.any():
    print("\n" + "="*60)
    print("🛑 CRITICAL ERROR: MISSING VALUES FOUND")
    print("="*60)
    print("The following columns still contain NaNs:")
    print("-" * 30)
    print(X.isnull().sum()[X.isnull().sum() > 0])
    print("-" * 30)
    raise ValueError("Data Validation Failed: Input data is not clean. Please fix NaNs in the source file before clustering.")
else:
    print("\n✅ Data Quality Check Passed: No missing values found.")

y = master_df['Target_Baseline']


✅ Data Quality Check Passed: No missing values found.


In [7]:
# 3. Split Data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42, stratify=y)

# 4. Train Logistic Regression
# max_iter=1000 ensures it converges even with unscaled data
log_reg = LogisticRegression(max_iter=1000, random_state=42)
log_reg.fit(X_train, y_train)

# 5. Evaluate
y_pred = log_reg.predict(X_test)
y_prob = log_reg.predict_proba(X_test)[:, 1]

print("\n--- Baseline Results ---")
print(classification_report(y_test, y_pred))
print(f"Baseline ROC-AUC: {roc_auc_score(y_test, y_prob):.4f}")

# 6. Interpret Coefficients (Optional but great for HW)
# This shows directionality: Positive = Increases Risk, Negative = Decreases Risk
coeffs = pd.Series(log_reg.coef_[0], index=feature_cols).sort_values()
print("\n--- Feature Coefficients (Top Drivers) ---")
print(coeffs)


--- Baseline Results ---
              precision    recall  f1-score   support

           0       0.85      0.92      0.89       404
           1       0.69      0.52      0.59       135

    accuracy                           0.82       539
   macro avg       0.77      0.72      0.74       539
weighted avg       0.81      0.82      0.81       539

Baseline ROC-AUC: 0.8656

--- Feature Coefficients (Top Drivers) ---
feat_subsidy_coverage_ratio   -7.393034
feat_pct_aian                 -0.500975
feat_pct_black                -0.095079
feat_net_attrition            -0.013285
feat_pct_hispanic             -0.009649
feat_passive_renewal          -0.007159
avg_premium_before_APTC        0.011143
feat_pct_gold                  0.021579
feat_pct_FPL_250_400           0.023823
feat_pct_bronze                0.030790
feat_pct_asian                 0.031432
feat_market_newness            0.067956
feat_pct_age_55_over           0.087767
feat_pct_FPL_150_250           0.159933
feat_pct_FPL_100_1

C:\Users\liuh102\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\linear_model\_logistic.py:473: ConvergenceWarning: lbfgs failed to converge after 1000 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=1000).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
